In [1]:
!pip install google-genai

In [2]:
import os
from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [3]:
import google.generativeai as genai

In [4]:
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    if not GOOGLE_API_KEY:
        raise KeyError("Chave não encontrada nos Secrets do Colab.")
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Chave da API do Google configurada com sucesso a partir dos Secrets do Colab.")
except (ImportError, KeyError) as e:
    print(f"Secrets do Colab não disponíveis ou chave não encontrada ({e}). Tentando input manual.")
    # Se não estiver usando Secrets, ou como fallback, pode pedir via input (menos seguro):
    GOOGLE_API_KEY = input("Por favor, insira sua Google API Key: ")
    if GOOGLE_API_KEY:
        genai.configure(api_key=GOOGLE_API_KEY)
        print("Chave da API do Google configurada manualmente.")
    else:
        print("Nenhuma chave de API fornecida. A chamada à API falhará.")
        # Você pode querer encerrar o script aqui ou lidar com a ausência da chave

Chave da API do Google configurada com sucesso a partir dos Secrets do Colab.


In [5]:
def obter_sugestao_gemini(textura, sabor, tempo, pessoas):
    """
    Coleta as informações, monta o prompt e chama a API do Gemini.
    """
    if not GOOGLE_API_KEY: # Verifica se a chave foi configurada
        return "Erro: A chave da API do Google não foi configurada. Não é possível buscar a receita."

    # Monta o prompt para a API do Gemini
    prompt = (
        f"Crie uma receita de pão {sabor} e {textura} que se encaixe na seguinte descrição de tempo: '{tempo}'. "
        f"A receita deve ser para {pessoas} pessoa(s). "
        "Por favor, detalhe os ingredientes e o modo de preparo de forma clara e completa. "
        "Formate a resposta de maneira organizada, com seções para 'Ingredientes' e 'Modo de Preparo'. "
        "Se for um pão rápido (30min-1h), sugira algo que não precise de longas fermentações."
    )

    print("\n--- PROMPT ENVIADO PARA A API DO GEMINI ---")
    print(prompt)
    print("-------------------------------------------\n")

    try:
        # Inicializa o modelo generativo (ex: 'gemini-1.0-pro' ou 'gemini-1.5-flash')
        # Verifique os modelos disponíveis e mais adequados na documentação.
        model = genai.GenerativeModel('gemini-2.0-flash')

        # Gera o conteúdo
        response = model.generate_content(prompt)

        # Retorna o texto da resposta
        # A API pode retornar blocos de texto, então é bom verificar e concatenar se necessário.
        # Para respostas simples de texto, response.text geralmente é suficiente.
        # Para respostas mais complexas ou com streaming, você pode precisar iterar sobre response.parts
        if response.parts:
            receita_gerada = "".join(part.text for part in response.parts)
        else:
            # Fallback para response.text, embora com a API atual, parts seja mais comum
            receita_gerada = response.text if hasattr(response, 'text') else "A API não retornou um texto na resposta esperada."

    except Exception as e:
        receita_gerada = f"Erro ao contatar a API do Gemini: {e}"
        print(f"Ocorreu um erro: {e}")
        # Informações adicionais de depuração, se disponíveis no erro:
        if hasattr(e, 'response') and hasattr(e.response, 'prompt_feedback'):
            print(f"Feedback do prompt: {e.response.prompt_feedback}")
        if hasattr(e, 'response') and hasattr(e.response, 'candidates'):
            for candidate in e.response.candidates:
                print(f"Candidate: {candidate.finish_reason}, {candidate.safety_ratings}")


    return receita_gerada

In [6]:
print("--- PadarIA - Configuração da Receita ---")
textura_escolhida = input("Você quer que seu pão seja macio ou crocante? (macio / crocante): ").lower()
sabor_escolhido = input("Você quer um pão doce ou salgado? (doce / salgado): ").lower()
print("\nQuanto tempo você tem?")
print("1: Quero tudo pronto em 30min")
print("2: Tenho até 1h")
print("3: Tenho algumas horas")
print("4: Pode ficar pronto no dia seguinte")
tempo_opcao = input("Escolha uma opção de tempo (1-4): ")
tempos_map = {
"1": "Quero tudo pronto em 30min",
"2": "Tenho até 1h",
"3": "Tenho algumas horas",
"4": "Pode ficar pronto no dia seguinte"
}
tempo_disponivel = tempos_map.get(tempo_opcao, "Tenho algumas horas") # Padrão se inválido
numero_pessoas = input("Para quantas pessoas é a receita?: ")

# Validar entrada de pessoas (simples)
if not numero_pessoas.isdigit() or int(numero_pessoas) <= 0:
    print("Número de pessoas inválido. Usando '1' como padrão.")
    numero_pessoas = "1"

# Passo 5: Chamar a função e imprimir o resultado
if GOOGLE_API_KEY: # Só tenta obter a receita se a chave estiver configurada
    print(f"\nBuscando receita para pão {sabor_escolhido}, {textura_escolhida}, para {numero_pessoas} pessoa(s), com tempo: {tempo_disponivel}...")
    receita_sugerida = obter_sugestao_gemini(textura_escolhida, sabor_escolhido, tempo_disponivel, numero_pessoas)

    print("\n--- RECEITA SUGERIDA PELA API DO GEMINI ---")
    print(receita_sugerida)
    print("---------------------------------------------")
else:
    print("\nNão foi possível buscar a receita pois a chave da API não foi configurada.")

--- PadarIA - Configuração da Receita ---
Você quer que seu pão seja macio ou crocante? (macio / crocante): crocante
Você quer um pão doce ou salgado? (doce / salgado): doce

Quanto tempo você tem?
1: Quero tudo pronto em 30min
2: Tenho até 1h
3: Tenho algumas horas
4: Pode ficar pronto no dia seguinte
Escolha uma opção de tempo (1-4): 1
Para quantas pessoas é a receita?: 1

Buscando receita para pão doce, crocante, para 1 pessoa(s), com tempo: Quero tudo pronto em 30min...

--- PROMPT ENVIADO PARA A API DO GEMINI ---
Crie uma receita de pão doce e crocante que se encaixe na seguinte descrição de tempo: 'Quero tudo pronto em 30min'. A receita deve ser para 1 pessoa(s). Por favor, detalhe os ingredientes e o modo de preparo de forma clara e completa. Formate a resposta de maneira organizada, com seções para 'Ingredientes' e 'Modo de Preparo'. Se for um pão rápido (30min-1h), sugira algo que não precise de longas fermentações.
-------------------------------------------


--- RECEITA SUG